<a href="https://colab.research.google.com/github/Aadhil21/DSGP-Group28/blob/branch-Aadhil/Extract_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Libraries

In [18]:
import pandas as pd
import cv2
import os
import mediapipe as mp

#Initialize mediapipe model

In [19]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(min_detection_confidence=0.6,
                    min_tracking_confidence=0.6,
                    static_image_mode=False,
                    smooth_landmarks=True,
                    model_complexity=2)

##Set Directory Path

In [20]:
if not os.path.exists('Videos'):
    os.mkdir('Videos')
if not os.path.exists('Datasets'):
    os.mkdir('Datasets')

video_name = 'PushUp.mp4'
video_path = 'Videos/' + video_name
dataset_path = 'Datasets/'

In [21]:
frame_count = 0
frame_numbers = []
data_dict = {}
desired_width = 480
desired_height = 360

In [22]:
cap = cv2.VideoCapture(video_path)

##Populate keypoints dictionary

In [23]:
for landmark in mp_pose.PoseLandmark:
    x_list_name = f"{landmark.name}_Point_x"
    y_list_name = f"{landmark.name}_Point_y"
    z_list_name = f"{landmark.name}_Point_z"
    visibility_list_name = f"{landmark.name}_visibility"

    data_dict[x_list_name] = []
    data_dict[y_list_name] = []
    data_dict[z_list_name] = []
    data_dict[visibility_list_name] = []

##Process Frames

In [25]:
while cap.isOpened():
    success, image = cap.read()
    if not success:
        break

    # Increment frame_count
    frame_count += 1
    print(f'Processing frame {frame_count}')

    # Resize the frame to the desired width and height
    image = cv2.resize(image, (desired_width, desired_height))

    # Convert the image from BGR to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    results = pose.process(image_rgb)

    if results.pose_landmarks:

        frame_numbers.append(frame_count)

        for landmark in mp_pose.PoseLandmark:
            x_list_name = f"{landmark.name}_Point_x"
            y_list_name = f"{landmark.name}_Point_y"
            z_list_name = f"{landmark.name}_Point_z"
            visibility_list_name = f"{landmark.name}_visibility"

            data_dict[x_list_name].append(results.pose_landmarks.landmark[landmark].x)
            data_dict[y_list_name].append(results.pose_landmarks.landmark[landmark].y)
            data_dict[z_list_name].append(results.pose_landmarks.landmark[landmark].z)
            data_dict[visibility_list_name].append(results.pose_landmarks.landmark[landmark].visibility)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

Processing frame 2


C:\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


AttributeError: confidence

## Save Key points to File

In [ ]:
# Creating a DataFrame from the dictionary
df = pd.DataFrame(data_dict)

# Adding a column for the frame number
df.insert(0, 'Frame_Number', frame_numbers)

# Writing the DataFrame to an Excel file
file_name = 'pose_landmark_data_' + video_name.split('.')[0] + '.xlsx'

df.to_excel(dataset_path + '/' + file_name, index=False)

print(f"Data saved to {file_name}")

## Reading Key points from File

In [ ]:
df_excel = pd.read_excel('Datasets/' + file_name)
df_excel